# Tutorial 2: Parameter & Node Management

## Prerequisite

- Follow the tutorial 1 and understand the main difference between param and node.

## Overview

Parameters and nodes allow users to impact a flow. Parameters can be **init parameters** (supplied during initialization) or **run parameters** (supplied during flow execution). Their values can be supplied by developers, or are automatically calculated based on other values. Nodes are usually supplied during initialization but can be nested in multiple levels.

In [1]:
%load_ext autoreload
%autoreload 2

from finestflow.base import Composable, Param, Node, param, node

## Declare params

Parameters can be declared in a flow as class-level attributes, subject to 2 constraints:

- Parameter name cannot starts with underscore `_`. If it starts with underscore, `finenestflow` will not recognize it as flow parameter and it will not show up in the list flow parameters.
- Parameter cannot be type annotated as `Composable` or subclass of `Composable`, since they will be treated as node, rather than parameter.

Similar to class-level attributes, setting the default level in class will allow the function to have that same value if not supplied.

In [13]:
class Sum(Composable):
    a: int
    b: int = 10
    c: int = 10

    def run(self) -> int:
        return self.a + self.b + self.c

step1 = Sum(a=10, b=20)
print(f"{step.a=}; {step.b=}; {step.c=}")
print(f"{step()=}")
print("-" * 10)

step.a=1; step.b=10; step.c=20
step()=31
----------


`finestflow` allows users to better fine-grainedly declare the param by explicitly use the `Param(...)` construct. It has the following options:

- `default=`: declare the default value directly (similar effect with example above).
- `default_callback=`: set the default value using callback function.
- `help=`: provide human-understandable description of the parameter that can help users easier understand the purpose or how to use the parameter.
- `refresh_on_set=True/False`: tell `finestflow` to refresh the function whenever this parameter is set.
- `strict_type=True/False`: tell `finestflow` to always make sure that the value supplied to this parameter has the correct type (requires the param to have type annotation).
- `depends_on=`: tell `finestflow` that the value of this param depends on the value of other params/nodes, and will recalculate the value of this param whenever those other params/nodes change (requires setting `default_callback`).

Computed params can also be declared using the `@param` decorator to mark the default callback function:

In [18]:
class Sum(Composable):
    a: int = 1
    b: int = Param(default=2)
    c: int = Param(default_callback=lambda obj, type_: obj.b * 2, depends_on=["b"])

    @param(depends_on=["c"])
    def d(self):
        return self.c * 3

    def run(self) -> int:
        return self.a + self.b + self.c + self.d

step = Sum()
print(f"{step.a=}; {step.b=}; {step.c=}; {step.d=}")
print(f"a + b + c + d: {step()=}")
print()
print("`c` depends on `b`, `d` depends on `c` After changing `b` = 10, observe the change")
step.b = 10
print(f"{step.a=}; {step.b=}; {step.c=}; {step.d=}")
print(f"a + b + c + d: {step()=}")
print("-" * 10)

step.a=1; step.b=2; step.c=4; step.d=12
a + b + c + d: step()=19

`c` depends on `b`, `d` depends on `c` After changing `b` = 10, observe the change
step.a=1; step.b=10; step.c=20; step.d=60
a + b + c + d: step()=91
----------


## Declare nodes

Nodes can be declared using `Composable` type annotation. However, to declare default value for a node, the user should use `Node(...)` construct to lazily initiate the node. Otherewise, that node will be created as class value, which can unnecessarily be resource intensive. The `Node(...)` construct has following options:

- `default=`: the node Composable class to initiate the node as default if the node is not set.
- `default_kwargs=`: a dictionary containing key-value params/nodes to use to supply initiate the default node (requires setting `default`).
- `default_callback=`: construct default node if the node is not set.
- `help=`: provide human-understandable description of the node that can help users easier understand the purpose of this node.
- `depends_on=`: tell `finestflow` that this node depends on the value of other params/nodes, and will recalculate the value of this node whenever those other params/nodes change (requires setting `default_callback`)
- `input=`: declare the input type annotation of this node execution (if this is blank, and the `default` is set with a Composable class that has input type annotation for its `run` method, then it will automatically supplied here).
- `output=`: declare the output type annotation of this node execution (can be auto set if this is blank, and the `default` is set with a Composable class that has output type annotation for its `run` method).

Computed nodes can also be declared using the `@node` decorator to mark the default callback function:

In [35]:
class Plus(Composable):
    a: int
    x: Composable
    y: Composable = Node(default=Sum, default_kwargs={"a": 100})

    @node(depends_on=["a"])
    def z(self):
        return Sum(a=self.a, b=1)
        
    def run(self) -> int:
        x, y, z = self.x(), self.y(), self.z()
        print(f"{x=}, {y=}, {z=}")
        return x + y + z

step = Plus(a=1, x=Sum(a=10))
print(f"{step()=}")
step.a = 2
print(f"{step()=}")
print("-" * 10)

x=28, y=118, z=10
step()=156
x=28, y=118, z=11
step()=157
----------


## Inspect params and nodes

As the flow grows more complex, knowing which params in which nodes hold which values can easy become a headache.

`finestflow` provides method to declare params and nodes. It also provides easy methods to inspect a flow:

- `.describe(original=True)`: return the nodes and params declared by a flow.
- `.describe(original=False)`: return the nodes and params with the current supplied values.
- `.missing()`: show missing params and nodes.

In [36]:
import yaml
step = Plus(a=1)

#### Return the nodes and params declared by a flow, contains the default values

In [37]:
print(yaml.dump(step.describe(original=True), sort_keys=False))
print("-" * 10)

type: __main__.Plus
params:
  a: null
nodes:
  y:
    type: __main__.Sum
    params:
      a: 1
      b: 2
    nodes: {}
  x: null

----------


#### Return the nodes and params with the current values

In [38]:
print(yaml.dump(step.describe(original=False), sort_keys=False))
print("-" * 10)

type: __main__.Plus
params:
  a: 1
nodes:
  x: null
  y:
    type: __main__.Sum
    params:
      a: 100
      b: 2
      c: 4
      d: 12
    nodes: {}
  z:
    type: __main__.Sum
    params:
      a: 1
      b: 1
      c: 2
      d: 6
    nodes: {}

----------


#### Return missing nodes and params in the pipelines

In [40]:
step = Plus()
missing = step.missing()
print("Params:", missing["params"])
print("Nodes:", missing["nodes"])
print("-" * 10)

Params: ['a']
Nodes: ['x']
----------


#### Return the list of params and nodes names

In [43]:
print(step.params)
print(step.nodes)

{'a': None}
['x', 'y', 'z']


## Set run-time parameters

In a complex flow, sometimes run-time parameters change very infrequently. Such run-time parameters appear not only in the root flow but also in nested flows. `finestflow` provide the `set_run(...)` method to set those run-time parameters once. The values set in these methods will persist across different runs of the pipeline. To just pass the params to nested flow without persisting it in different run, set `_ff_run_kwargs` when executing the pipeline.

In [52]:
class Add(Composable):
    a: int

    def run(self, b, c) -> int:
        return self.a + b + c


class Minus(Composable):
    a: int

    def run(self, b) -> int:
        return self.a - b


class Op1(Composable):
    x: Composable
    y: Composable

    def run(self, z):
        return self.x() + self.y() + z

op = Op1(x=Add(a=5), y=Minus(a=10))
op.set_run({"x.b": 100, "y.b": 5})
print(op(z=1, _ff_run_kwargs={"x.c": 1}))     # 5 + 100 + 1 + 10 - 5 + 1
print("-" * 10)

112
----------
